# **Homework 1: COVID-19 Cases Prediction (Regression) -1**

Objectives:
* Solve a regression problem with deep neural networks (DNN).
* Understand basic DNN training tips.
* Familiarize yourself with PyTorch.

If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to mlta-2022-spring@googlegroups.com

# Download data
If the Google Drive links below do not work, you can download data from [Kaggle](https://www.kaggle.com/c/ml2022spring-hw1/data), and upload data manually to the workspace.

In [1]:
!gdown --id '1kLSW_-cW2Huj7bh84YTdimGBOJaODiOS' --output covid.train.csv
!gdown --id '1iiI5qROrAhZn-o4FPqsE97bMzDEFvIdg' --output covid.test.csv

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1kLSW_-cW2Huj7bh84YTdimGBOJaODiOS
To: /content/covid.train.csv
100% 2.49M/2.49M [00:00<00:00, 185MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1iiI5qROrAhZn-o4FPqsE97bMzDEFvIdg
To: /content/covid.test.csv
100% 993k/993k [00:00<00:00, 151MB/s]


# Import packages

In [2]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# For Progress Bar
from tqdm import tqdm

# Pytorch
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

# Some Utility Functions

You do not need to modify this part.

In [3]:
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set)) 
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)                        
        with torch.no_grad():                   
            pred = model(x)                     
            preds.append(pred.detach().cpu())   
    preds = torch.cat(preds, dim=0).numpy()  
    return preds

# Data Preprocessing

In [4]:
df = pd.read_csv('./covid.train.csv')
df.head()

,id,AL,AK,AZ,AR,CA,CO,CT,FL,GA,...,work_outside_home.4,shop.4,restaurant.4,spent_time.4,large_event.4,public_transit.4,anxious.4,depressed.4,worried_finances.4,tested_positive.4
0,0,0,0,0,0,0,0,0,1,0,...,31.113209,67.394551,36.674291,40.743132,17.842221,4.093712,10.440071,8.627117,37.329512,7.456154
1,1,0,0,0,0,0,1,0,0,0,...,33.920257,64.398380,34.612238,44.035688,17.808103,4.924935,10.172662,9.954333,32.508881,8.010957
2,2,0,0,0,0,0,0,0,0,0,...,31.604604,62.101064,26.521875,36.746453,13.903667,7.313833,10.388712,7.956139,36.745588,2.906977
3,3,0,0,0,0,0,0,0,0,0,...,35.115738,67.935520,38.022492,48.434809,27.134876,3.101904,10.498683,8.231522,38.680162,12.575816
4,4,0,0,0,0,0,0,0,0,0,...,35.129714,69.934592,38.242368,49.095933,22.683709,4.594620,9.878927,9.469290,28.344123,21.428589


In [5]:
# Calculate Pearson Correlation
cor = df.corr()
# Correlation with output variable
cor_target = abs(cor["tested_positive.4"])
# Selecting highly correlated features
relevant_features = cor_target[cor_target > 0.8]  # 0.5

In [6]:
df = df[list(relevant_features.index)]

In [7]:
pd.set_option('display.max_columns', None)
df.head()

,cli,ili,hh_cmnty_cli,nohh_cmnty_cli,tested_positive,cli.1,ili.1,hh_cmnty_cli.1,nohh_cmnty_cli.1,tested_positive.1,cli.2,ili.2,hh_cmnty_cli.2,nohh_cmnty_cli.2,tested_positive.2,cli.3,ili.3,hh_cmnty_cli.3,nohh_cmnty_cli.3,tested_positive.3,cli.4,ili.4,hh_cmnty_cli.4,nohh_cmnty_cli.4,tested_positive.4
0,0.658466,0.724606,12.939284,8.556230,7.374846,0.653157,0.713249,12.488933,8.219380,7.219989,0.677783,0.737965,12.487637,8.135510,7.077938,0.666751,0.723506,12.367718,8.006131,7.452243,0.685628,0.740943,12.364307,8.151578,7.456154
1,0.693287,0.675962,15.008467,11.091818,9.850027,0.738029,0.720511,15.070049,10.990937,10.050547,0.831586,0.827523,14.568504,10.578924,10.388084,0.767643,0.763580,14.264862,10.212529,8.707858,0.713255,0.719378,12.894363,8.919288,8.010957
2,0.617041,0.617041,8.614719,5.238097,3.897851,0.663597,0.635459,8.472831,5.154184,3.083111,0.534394,0.497435,8.493353,5.125553,3.132834,0.536224,0.498305,8.227657,5.298912,3.444182,0.479111,0.432445,8.067909,5.333533,2.906977
3,1.193768,1.272051,12.007656,7.219741,10.297712,1.356577,1.432775,12.997184,7.843776,9.280636,1.719706,1.690806,14.010509,9.008978,12.146962,1.737550,1.709244,14.364735,9.096583,13.404256,1.793346,1.766201,14.383789,9.416096,12.575816
4,1.455489,1.455489,20.957729,16.829163,24.056627,1.298511,1.231606,20.189998,16.229791,17.727289,1.486851,1.420611,20.160351,15.672487,18.650809,1.373938,1.308346,20.103385,15.798003,20.238111,1.264465,1.198567,20.257355,15.864201,21.428589


In [8]:
len(df.columns)

25

In [9]:
df.shape

(2699, 25)

In [10]:
def concat_df(df_old, df_new):
    new_cols = {x: y for x, y in zip(df_new.columns, df_old.columns)}
    df_out = df_old.append(df_new.rename(columns=new_cols))
    return df_out

## Split Training Data (兩天一組)

In [11]:
df1 = df.iloc[:, :10]
df2 = df.iloc[:, 5:15]
df3 = df.iloc[:, 10:20]
df4 = df.iloc[:, 15:25]

In [12]:
print(df1.columns)
result1 = concat_df(df1, df2)
print(result1)

Index(['cli', 'ili', 'hh_cmnty_cli', 'nohh_cmnty_cli', 'tested_positive',
       'cli.1', 'ili.1', 'hh_cmnty_cli.1', 'nohh_cmnty_cli.1',
       'tested_positive.1'],
      dtype='object')
           cli       ili  hh_cmnty_cli  nohh_cmnty_cli  tested_positive  \
0     0.658466  0.724606     12.939284        8.556230         7.374846   
1     0.693287  0.675962     15.008467       11.091818         9.850027   
2     0.617041  0.617041      8.614719        5.238097         3.897851   
3     1.193768  1.272051     12.007656        7.219741        10.297712   
4     1.455489  1.455489     20.957729       16.829163        24.056627   
...        ...       ...           ...             ...              ...   
2694  1.171500  1.163963     23.816931       19.707350        11.047798   
2695  0.655551  0.660840     11.519438        7.919253         6.062231   
2696  1.728777  1.721787     29.207283       23.868097        13.859660   
2697  0.840583  0.829843     13.205292       10.390792        

In [13]:
result1 = concat_df(df1, df2)
result1 = concat_df(result1, df3)
result1 = concat_df(result1, df4)

In [14]:
print(result1.shape)
result1.head()

(10796, 10)


,cli,ili,hh_cmnty_cli,nohh_cmnty_cli,tested_positive,cli.1,ili.1,hh_cmnty_cli.1,nohh_cmnty_cli.1,tested_positive.1
0,0.658466,0.724606,12.939284,8.556230,7.374846,0.653157,0.713249,12.488933,8.219380,7.219989
1,0.693287,0.675962,15.008467,11.091818,9.850027,0.738029,0.720511,15.070049,10.990937,10.050547
2,0.617041,0.617041,8.614719,5.238097,3.897851,0.663597,0.635459,8.472831,5.154184,3.083111
3,1.193768,1.272051,12.007656,7.219741,10.297712,1.356577,1.432775,12.997184,7.843776,9.280636
4,1.455489,1.455489,20.957729,16.829163,24.056627,1.298511,1.231606,20.189998,16.229791,17.727289


## Split Test Data (兩天一組)

In [15]:
df_test = pd.read_csv('./covid.test.csv')
df_test = df_test[list(relevant_features.index)[:-1]]
df_test.head()

,cli,ili,hh_cmnty_cli,nohh_cmnty_cli,tested_positive,cli.1,ili.1,hh_cmnty_cli.1,nohh_cmnty_cli.1,tested_positive.1,cli.2,ili.2,hh_cmnty_cli.2,nohh_cmnty_cli.2,tested_positive.2,cli.3,ili.3,hh_cmnty_cli.3,nohh_cmnty_cli.3,tested_positive.3,cli.4,ili.4,hh_cmnty_cli.4,nohh_cmnty_cli.4
0,0.710274,0.724032,14.561642,11.117583,7.465515,0.655787,0.670090,14.312849,10.883053,7.561836,0.779843,0.802076,15.217542,11.623847,8.423512,0.765527,0.788473,15.840262,12.142336,8.357021,0.692846,0.712477,15.936616,12.241230
1,0.347450,0.346714,9.727842,6.300419,6.807512,0.462280,0.453509,10.216577,6.968233,6.490388,0.515629,0.506906,10.497930,7.155311,6.872041,0.687033,0.697734,11.163712,7.868932,9.318186,0.668739,0.679464,10.995437,8.030379
2,0.655551,0.660840,11.519438,7.919253,6.062231,0.635128,0.635128,11.514569,8.035914,5.181086,0.563209,0.575010,11.500600,8.295961,5.032925,0.605297,0.608920,11.135534,8.133172,5.040143,0.588866,0.592042,11.414773,8.389935
3,0.877624,0.924874,18.057030,14.794566,9.069323,0.904217,0.943838,18.314449,15.116528,9.224887,0.962423,1.011297,18.647339,15.350428,9.365114,0.832804,0.886137,18.813264,15.625344,8.987720,0.849647,0.903111,18.951776,15.802525
4,1.632451,1.642431,26.821549,22.107107,18.982016,1.468624,1.478914,26.009325,20.996176,19.349288,1.475079,1.487395,26.059658,21.072372,18.039932,1.458864,1.496418,25.181046,20.159494,17.321401,1.317712,1.345097,24.282915,19.062190


In [16]:
df_test.shape

(1078, 24)

In [17]:
df_test_1 = df_test.iloc[:, :10]
df_test_2 = df_test.iloc[:, 5:15]
df_test_3 = df_test.iloc[:, 10:20]

In [18]:
result_test_1 = concat_df(df_test_1, df_test_2)
result_test_1 = concat_df(result_test_1, df_test_3)

In [19]:
print(result_test_1.shape)
result_test_1.head()

(3234, 10)


,cli,ili,hh_cmnty_cli,nohh_cmnty_cli,tested_positive,cli.1,ili.1,hh_cmnty_cli.1,nohh_cmnty_cli.1,tested_positive.1
0,0.710274,0.724032,14.561642,11.117583,7.465515,0.655787,0.670090,14.312849,10.883053,7.561836
1,0.347450,0.346714,9.727842,6.300419,6.807512,0.462280,0.453509,10.216577,6.968233,6.490388
2,0.655551,0.660840,11.519438,7.919253,6.062231,0.635128,0.635128,11.514569,8.035914,5.181086
3,0.877624,0.924874,18.057030,14.794566,9.069323,0.904217,0.943838,18.314449,15.116528,9.224887
4,1.632451,1.642431,26.821549,22.107107,18.982016,1.468624,1.478914,26.009325,20.996176,19.349288


## 合併產生新的訓練資料

In [20]:
result_train_test_1 = concat_df(result1, result_test_1)
result_train_test_1.to_csv("./covid.train_test_1.csv", index=False)

In [21]:
result_train_test_1.shape

(14030, 10)

In [22]:
result_train_test_1.tail()

,cli,ili,hh_cmnty_cli,nohh_cmnty_cli,tested_positive,cli.1,ili.1,hh_cmnty_cli.1,nohh_cmnty_cli.1,tested_positive.1
1073,4.810023,4.930284,52.742005,46.223098,33.549062,5.056160,5.164526,52.635399,45.633182,34.868878
1074,5.171595,5.361401,49.445407,41.666937,34.463884,5.207976,5.388358,49.523259,41.609476,34.440193
1075,4.269012,4.521401,47.779926,40.474252,36.884642,4.210177,4.504812,48.089471,40.594459,36.530228
1076,5.437081,5.583710,51.424241,43.586619,35.831536,5.519345,5.653417,51.281594,43.569794,35.944579
1077,4.777574,4.982530,50.017690,42.755661,39.599902,4.837186,5.030315,50.391041,43.481522,39.724689


## 產生新的測試資料

In [23]:
df_test = pd.read_csv('./covid.test.csv')
df_test.head()

,id,AL,AK,AZ,AR,CA,CO,CT,FL,GA,ID,IL,IN,IA,KS,KY,LA,MD,MA,MI,MN,MS,MO,NE,NV,NJ,NM,NY,NC,OH,OK,OR,RI,SC,TX,UT,VA,WA,cli,ili,hh_cmnty_cli,nohh_cmnty_cli,wearing_mask,travel_outside_state,work_outside_home,shop,restaurant,spent_time,large_event,public_transit,anxious,depressed,worried_finances,tested_positive,cli.1,ili.1,hh_cmnty_cli.1,nohh_cmnty_cli.1,wearing_mask.1,travel_outside_state.1,work_outside_home.1,shop.1,restaurant.1,spent_time.1,large_event.1,public_transit.1,anxious.1,depressed.1,worried_finances.1,tested_positive.1,cli.2,ili.2,hh_cmnty_cli.2,nohh_cmnty_cli.2,wearing_mask.2,travel_outside_state.2,work_outside_home.2,shop.2,restaurant.2,spent_time.2,large_event.2,public_transit.2,anxious.2,depressed.2,worried_finances.2,tested_positive.2,cli.3,ili.3,hh_cmnty_cli.3,nohh_cmnty_cli.3,wearing_mask.3,travel_outside_state.3,work_outside_home.3,shop.3,restaurant.3,spent_time.3,large_event.3,public_transit.3,anxious.3,depressed.3,worried_finances.3,tested_positive.3,cli.4,ili.4,hh_cmnty_cli.4,nohh_cmnty_cli.4,wearing_mask.4,travel_outside_state.4,work_outside_home.4,shop.4,restaurant.4,spent_time.4,large_event.4,public_transit.4,anxious.4,depressed.4,worried_finances.4
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.710274,0.724032,14.561642,11.117583,52.905815,17.842008,31.733328,66.964202,31.863729,41.563975,18.757929,4.101058,11.745555,9.200281,31.232455,7.465515,0.655787,0.670090,14.312849,10.883053,53.632802,18.392039,32.009796,66.676857,31.912861,42.133776,18.800122,3.878775,11.858192,9.127592,30.700602,7.561836,0.779843,0.802076,15.217542,11.623847,55.007192,18.497708,33.000687,66.215625,31.782611,42.397610,18.760560,3.745753,12.902437,9.538124,31.337024,8.423512,0.765527,0.788473,15.840262,12.142336,56.413262,19.153466,32.686673,65.945659,31.699248,42.274692,18.465502,3.623784,13.008737,9.857105,31.347696,8.357021,0.692846,0.712477,15.936616,12.241230,57.509701,18.463415,32.470622,65.236954,31.730028,42.031183,18.188452,3.472105,12.452570,9.400024,30.550873
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.347450,0.346714,9.727842,6.300419,49.686001,22.680990,31.599276,61.308271,29.826095,40.833147,17.480809,6.674493,10.628595,6.729862,32.249644,6.807512,0.462280,0.453509,10.216577,6.968233,51.024665,23.056488,31.678939,61.180916,28.753107,40.665370,17.015020,6.461184,10.898060,7.044703,32.983522,6.490388,0.515629,0.506906,10.497930,7.155311,52.208179,24.360387,32.102060,60.163167,28.494685,40.704227,16.562653,6.364917,11.206494,7.223559,33.963705,6.872041,0.687033,0.697734,11.163712,7.868932,54.315909,23.572758,32.419781,60.108387,28.194897,40.136268,16.567779,6.162729,11.376842,7.155867,34.603368,9.318186,0.668739,0.679464,10.995437,8.030379,54.808565,24.201702,31.499036,60.360030,28.319417,39.919723,16.614292,5.992332,11.747528,7.438606,35.364871
2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.655551,0.660840,11.519438,7.919253,50.658992,13.276638,28.923377,64.429955,30.377334,40.047315,16.241631,13.864530,12.905281,9.569775,37.417768,6.062231,0.635128,0.635128,11.514569,8.035914,51.245370,13.452431,28.578727,64.871154,30.588031,40.146444,16.352592,14.048134,13.061025,10.005939,37.606251,5.181086,0.563209,0.575010,11.500600,8.295961,51.644114,13.537471,29.117064,65.779317,31.079145,40.073858,16.539901,13.795196,13.043017,10.334899,36.776059,5.032925,0.605297,0.608920,11.135534,8.133172,52.486643,13.804978,28.984491,66.126427,31.299807,40.399527,16.783327,14.058799,13.129031,10.492896,36.670282,5.040143,0.588866,0.592042,11.414773,8.389935,53.322176,13.742657,28.696996,66.114428,31.211381,40.448538,16.998722,13.951631,12.898089,10.214711,37.028705
3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.877624,0.924874,18.057030,14.794566,61.153680,13.903389,27.536333,65.410381,29.107429,41.124830,13.881023,5.055974,14.355803,10.489910,31.290344,9.069323,0.904217,0.943838,18.314449,15.116528,61.70565

In [24]:
df_test = df_test[list(relevant_features.index)[:-1]]
df_test.head()

,cli,ili,hh_cmnty_cli,nohh_cmnty_cli,tested_positive,cli.1,ili.1,hh_cmnty_cli.1,nohh_cmnty_cli.1,tested_positive.1,cli.2,ili.2,hh_cmnty_cli.2,nohh_cmnty_cli.2,tested_positive.2,cli.3,ili.3,hh_cmnty_cli.3,nohh_cmnty_cli.3,tested_positive.3,cli.4,ili.4,hh_cmnty_cli.4,nohh_cmnty_cli.4
0,0.710274,0.724032,14.561642,11.117583,7.465515,0.655787,0.670090,14.312849,10.883053,7.561836,0.779843,0.802076,15.217542,11.623847,8.423512,0.765527,0.788473,15.840262,12.142336,8.357021,0.692846,0.712477,15.936616,12.241230
1,0.347450,0.346714,9.727842,6.300419,6.807512,0.462280,0.453509,10.216577,6.968233,6.490388,0.515629,0.506906,10.497930,7.155311,6.872041,0.687033,0.697734,11.163712,7.868932,9.318186,0.668739,0.679464,10.995437,8.030379
2,0.655551,0.660840,11.519438,7.919253,6.062231,0.635128,0.635128,11.514569,8.035914,5.181086,0.563209,0.575010,11.500600,8.295961,5.032925,0.605297,0.608920,11.135534,8.133172,5.040143,0.588866,0.592042,11.414773,8.389935
3,0.877624,0.924874,18.057030,14.794566,9.069323,0.904217,0.943838,18.314449,15.116528,9.224887,0.962423,1.011297,18.647339,15.350428,9.365114,0.832804,0.886137,18.813264,15.625344,8.987720,0.849647,0.903111,18.951776,15.802525
4,1.632451,1.642431,26.821549,22.107107,18.982016,1.468624,1.478914,26.009325,20.996176,19.349288,1.475079,1.487395,26.059658,21.072372,18.039932,1.458864,1.496418,25.181046,20.159494,17.321401,1.317712,1.345097,24.282915,19.062190


In [25]:
df_test1 = df_test.iloc[:, -9:]
df_test1.head()

,cli.3,ili.3,hh_cmnty_cli.3,nohh_cmnty_cli.3,tested_positive.3,cli.4,ili.4,hh_cmnty_cli.4,nohh_cmnty_cli.4
0,0.765527,0.788473,15.840262,12.142336,8.357021,0.692846,0.712477,15.936616,12.241230
1,0.687033,0.697734,11.163712,7.868932,9.318186,0.668739,0.679464,10.995437,8.030379
2,0.605297,0.608920,11.135534,8.133172,5.040143,0.588866,0.592042,11.414773,8.389935
3,0.832804,0.886137,18.813264,15.625344,8.987720,0.849647,0.903111,18.951776,15.802525
4,1.458864,1.496418,25.181046,20.159494,17.321401,1.317712,1.345097,24.282915,19.062190


In [26]:
df_test1.to_csv("./covid.test_1.csv", index=False)

# Dataset

In [27]:
class COVID19Dataset(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)
        
    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

# Neural Network Model
Try out different model architectures by modifying the class below.

In [28]:
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        # TODO: modify model's structure, be aware of dimensions. 
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 256),
            # nn.LeakyReLU(),
            # nn.BatchNorm1d(256),
            nn.Dropout(0.4),            
            nn.Linear(256, 1),
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1) # (B, 1) -> (B)
        return x

# Feature Selection
Choose features you deem useful by modifying the function below.

In [29]:
def select_feat(train_data, valid_data, test_data, select_all=True, feature_selection_idx_list=None):
    '''Selects useful features to perform regression'''
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:,:-1], valid_data[:,:-1], test_data

    if not select_all and feature_selection_idx_list is not None:
        feat_idx = feature_selection_idx_list
    else:
        feat_idx = list(range(raw_x_train.shape[1]))
        
    return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid

# Training Loop

In [30]:
def trainer(train_loader, valid_loader, model, config, device):

    criterion = nn.MSELoss(reduction='mean') # Define your loss function, do not modify this.

    # Define your optimization algorithm. 
    # TODO: Please check https://pytorch.org/docs/stable/optim.html to get more available algorithms.
    # TODO: L2 regularization (optimizer(weight decay...) or implement by your self).
    optimizer = torch.optim.AdamW(model.parameters(), weight_decay=5e-5)  # 5e-5
#     optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.5) 

    writer = SummaryWriter() # Writer of tensoboard.

    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in range(n_epochs):
        model.train() # Set your model to train mode.
        loss_record = []

        # tqdm is a package to visualize your training progress.
        train_pbar = tqdm(train_loader, position=0, leave=True)

        for x, y in train_pbar:
            optimizer.zero_grad()               # Set gradient to zero.
            x, y = x.to(device), y.to(device)   # Move your data to device. 
            pred = model(x)             
            loss = criterion(pred, y)  # RMSE
            loss.backward()                     # Compute gradient(backpropagation).
            optimizer.step()                    # Update parameters.
            step += 1
            loss_record.append(loss.detach().item())
            
            # Display current epoch number and loss on tqdm progress bar.
            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})

        mean_train_loss = sum(loss_record)/len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval() # Set your model to evaluation mode.
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())
            
        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            print(f'Best Loss: {best_loss}')
            return

# Configurations
`config` contains hyper-parameters for training and the path to save your model.

In [31]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# torch.cuda.set_device(1) 
# device = torch.device("cuda", 1)
print(device)

config = {
    'seed': 5201314,      # 5201314 Your seed number, you can pick your lucky number. :)
    'select_all': True,   # True Whether to use all features.
    'valid_ratio': 0.1, # 0.3   # validation_size = train_size * valid_ratio
    'n_epochs': 5000,     # 5000 Number of epochs.            
    'batch_size': 2048,  # 1024
    'learning_rate': 2e-4, # 2e-4             
    'early_stop': 300,   # 300 If model has not improved for this many consecutive epochs, stop training.     
    'save_path': './models/model_train_test_73_1.ckpt'  # Your model will be saved here.
}

cuda


# Dataloader
Read data from files and set up training, validation, and testing sets. You do not need to modify this part.

In [32]:
# Set seed for reproducibility
same_seed(config['seed'])


# train_data size: 2699 x 118 (id + 37 states + 16 features x 5 days) 
# test_data size: 1078 x 117 (without last day's positive rate)
train_data, test_data = pd.read_csv('./covid.train_test_1.csv').values, pd.read_csv('./covid.test_1.csv').values
train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])

# Print out the data size before feature selection.
print(f""" -- Before feature selection --
train_data size: {train_data.shape} 
valid_data size: {valid_data.shape} 
test_data size: {test_data.shape}""")

# Select features
x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['select_all'])

# Print out the data size after feature selection.
print(f""" -- After feature selection --
train_data size: {x_train.shape} 
valid_data size: {x_valid.shape} 
test_data size: {x_test.shape}""")

# Print out the number of features.
print(f'number of features: {x_train.shape[1]}')

train_dataset = COVID19Dataset(x_train, y_train)
valid_dataset = COVID19Dataset(x_valid, y_valid)
test_dataset = COVID19Dataset(x_test)
                                            
# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

 -- Before feature selection --
train_data size: (12627, 10) 
valid_data size: (1403, 10) 
test_data size: (1078, 9)
 -- After feature selection --
train_data size: (12627, 9) 
valid_data size: (1403, 9) 
test_data size: (1078, 9)
number of features: 9


In [33]:
# print(relevant_features)

In [34]:
# train_data size: (2160, 118) 
# valid_data size: (539, 118) 
# test_data size: (1078, 117)
# number of features: 117

# Start training!

In [35]:
model = My_Model(input_dim=x_train.shape[1]).to(device) # put your model and data on the same computation device.
trainer(train_loader, valid_loader, model, config, device)

Epoch [1/5000]: 100%|██████████| 7/7 [00:02<00:00,  2.80it/s, loss=28.1]


Epoch [1/5000]: Train loss: 101.2331, Valid loss: 9.9273
Saving model with loss 9.927...


Epoch [2/5000]: 100%|██████████| 7/7 [00:00<00:00, 33.51it/s, loss=50.2]


Epoch [2/5000]: Train loss: 37.6513, Valid loss: 32.6628


Epoch [3/5000]: 100%|██████████| 7/7 [00:00<00:00, 32.54it/s, loss=22.5]


...

Epoch [2251/5000]: Train loss: 1.3770, Valid loss: 1.1761


Epoch [2252/5000]: 100%|██████████| 7/7 [00:00<00:00, 43.46it/s, loss=1.45]


Epoch [2252/5000]: Train loss: 1.4248, Valid loss: 1.2049


Epoch [2253/5000]: 100%|██████████| 7/7 [00:00<00:00, 17.88it/s, loss=1.46]

Epoch [2253/5000]: Train loss: 1.4281, Valid loss: 1.1238

Model is not improving, so we halt the training session.
Best Loss: 1.0371233224868774


In [36]:
# Best Valid loss without FS: 2.4238
# Best Valid loss with FS: 1.1371
# Best Valid loss with Adam: 1.0121
# Best Valid loss with AdamW: 0.9102
# Current Best: 0.9029

# 0.8854

# Plot learning curves with `tensorboard` (optional)

`tensorboard` is a tool that allows you to visualize your training progress.

If this block does not display your learning curve, please wait for few minutes, and re-run this block. It might take some time to load your logging information. 

In [37]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

<IPython.core.display.Javascript object>

# Testing
The predictions of your model on testing set will be stored at `pred.csv`.

In [38]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

model = My_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device) 
save_pred(preds, 'pred.csv')         

100%|██████████| 1/1 [00:00<00:00, 139.89it/s]


In [39]:
preds

array([ 8.321108,  9.14913 ,  5.192459, ..., 37.010036, 36.64289 ,
       39.66888 ], dtype=float32)

In [40]:
# np.save('pred_arr_train_test_1', preds)

# Reference
This notebook uses code written by Heng-Jui Chang @ NTUEE (https://github.com/ga642381/ML2021-Spring/blob/main/HW01/HW01.ipynb)